## Example of the Dynamic Mode Decomposition (DMD) of a 1D single wave 
Test Function : 
$$
f(x,t)=f_{1}(x,t)+f_{2}(x,t)=sech(x+3)\exp(i2.3t)+2sech(x)\tanh(x)\exp(i2.8t)
$$

In [11]:
using LinearAlgebra, Plots;gr();default(size=(300,200))

┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1278


In [ ]:
# Define the dataset Y
Nx = 400
x = range(-10,10,length=Nx) #  spatial coordinate (dx = 0.01)

Nt = 201
t = range(0,4π,length=Nt) # time coordinate (dt = 0.01)
dt=.01;

In [ ]:
#  Building the two dimensional continous function f(x,t)
f_1(x,t)=sech(x+3)*exp(im*2.3*t)
f_2(x,t)=2*sech(x)*tanh(x)exp(im*2.8*t)
f(x,t)=f_1(x,t)+f_2(x,t)

In [ ]:
anim=Animation()
anim = @animate for tt in t
    plot( x, real(f_1.(x,tt)),title="f_1 for two seconds",xlim=(-10,10),ylim=(-1.,1.),leg=false,xlabel="x",ylabel="f_1(x,t)")
end
gif(anim, "img/DMD2_f_1_fps15.gif", fps = 15)

In [ ]:
anim=Animation()
anim = @animate for tt in t
    plot( x, real(f_2.(x,tt)),xlim=(-10,10),ylim=(-1,1),title="f_2 for two seconds",leg=false,xlabel="x",ylabel="f_2(x,t)")
end
gif(anim, "img/DMD2_f_2_fps15.gif", fps = 15)

In [ ]:
anim=Animation()
anim = @animate for tt in t
    plot( x, real(f.(x,tt)),xlim=(-10,10),ylim=(-2,2),title="f for two seconds",leg=false,xlabel="x",ylabel="f(x,t)")
end
gif(anim, "img/DMD2_f_fps15.gif", fps = 15)

In [ ]:
# 2D Contour Plot of the data matrix Y
contour(x,t,f,levels=20)
xlabel!("x")
ylabel!("t")
title!("Contour Plot")

In [ ]:
# 3D Surface Plot of the dataset
#my_cg = cgrad([:blue, :yellow])
surface(x,t,f, xaxis="x", yaxis="t",zaxis="Y", title="Surface Plot")

In [ ]:
#  Building the discrete spatio-temporal series Y 
#Build of the spatio-temporal x vs t data matrix Y
Y = Complex.(zeros(Nx,Nt))
j=1
for tt in t
    Y[:,j] = real(f.(x,tt))  
    j+=1                                           
end
println("\rThe Data Matrix Y = \r")
Y

In [ ]:
# another way to create the data matrix Y (like the matlab meshgrid function )
X = repeat(reshape(x, :, 1), 1,length(t))

In [ ]:
T = repeat(reshape(t,1,:),  length(x),1)

In [ ]:
YY = map(f, X, T)

In [ ]:
# 2D Contour Plot of the data matrix Y
contour(X[:,1],T[1,:],real(YY)',levels=20)
xlabel!("x")
ylabel!("t")
title!("Contour Plot")

In [ ]:
surface(X[:,1],T[1,:],real(YY)', xaxis="x", yaxis="t",zaxis="YY", title="Surface Plot")

In [ ]:
#= Interactive Plot 
using PlotlyJS
trace=PlotlyJS.surface(x=x,y=t,z=real(Y), xlabel="x",colorscale="Viridis")
layout=PlotlyJS.Layout(title="Interactive Plot")
PlotlyJS.plot(trace,layout) =#

In [ ]:
# Data decomposition with the singular value decomposition 
# U matrix (spatial Correlations); Σ matrix (weighting of projections), Vt (time dynamics)
Y1 = Y[:,1:end - 1]

In [ ]:
Y2 = Y[:,2:end]

In [ ]:
U, Σ, V=svd(Y1)

In [ ]:
#= plot of the singular values (see the  two most dominants features due to the two ranks dataset 
(the two bumps with their temporal dynamics) The very good decay shows the existence of a low rank subspace =#
plot(Σ, yaxis=:log, xlim=(0,10),shape =:circle)

In [ ]:
# Plot of the POD modes (Uj ; j={1,2}). Due to the very differents spatial scales, the POD modes extracted have a physical meanng 
plot(x,U[:,1])
plot!(x,U[:,2])
xlabel!("x")
ylabel!("u1(x), u2(x)")
title!("POD modes") # Spatial modes 

In [ ]:
#plot of the Mode coefficents (σjVtj ; j={1,2]})
plot(0:0.01:1.99,Σ[1]*V[:,1],xlim=(0, 1))
plot!(0:0.01:1.99,Σ[2]*V[:,2],xlim=(0, 1))
xlabel!("t");ylabel!("a1(t), a2(t)");title!("Mode coefficients") # Temporal modes

In [ ]:
# SVD and rank-2 truncation
r = 2
Uᵣ = U[:,1:r]
Σᵣ = Diagonal(Σ[1:r])
Vᵣ = V[:,1:r];

In [ ]:
# Build Ã and DMD Modes
Ã = Uᵣ' * Y2 * Vᵣ / Σᵣ
Λ, W = eigen(Ã, sortby=nothing)
Φ = Y2 * Vᵣ / Σᵣ * W

In [ ]:
# DMD Spectra
omega = log.(Λ)/dt

In [ ]:
# DMD mode amplitude
y₁=Y1[:,1]
b=Φ\y₁

In [ ]:
for i in 1:length(t)
    print(t[i])
end 

In [ ]:
times_dynamics=zeros(r,length(t))

In [ ]:
for i in 1:length(t)
    time_dynamics[:,i] = b.*t[i]
end    

In [ ]:
x₂=Φ*Diagonal(Λ)*b

## Reduction and Reconstruction with feature extraction (two building blocks with their own dynamics are shown to exist)

In [ ]:
Φ[:,1]*Λ[1]*V[:,1]'

In [ ]:
#Animation of the POD1 mode over the first second
Y_DMD1=real(Φ[:,1]*Λ[1]*V[:,1]')
anim=Animation()
j=1
anim = @animate for tt in range(0.01,1,length=100)
    plot( x, Y_DMD1[:,j],leg=false,xlim=(-2,2),ylim=(-0.02,0.02),title="DMD1 mode",xlabel="x",ylabel="Y_DMD1")
    j+=1
end
gif(anim, "img/DMD_Y_DMD1_fps15.gif", fps = 15)

In [ ]:
#Animation of the POD2 mode over the first second
Y_DMD2=real(Φ[:,2]*Λ[2]*V[:,2]')
anim=Animation()
j=1
anim = @animate for tt in range(0.01,1,length=100)
    plot( x, Y_DMD2[:,j],leg=false,xlim=(-2,2),ylim=(-0.02,0.02),title="DMD2 mode",xlabel="x",ylabel="Y_DMD2")
    j+=1
end
gif(anim, "img/DMD_Y_DMD2_fps15.gif", fps = 15)

In [ ]:
#Animation of Yrecons = Y_POD1+YPOD2 over the firt second
Yrecons=Y_DMD1+Y_DMD2
anim=Animation()
j=1
anim = @animate for tt in range(0.01,1,length=100)
    plot( x, Yrecons[:,j],xlim=(-2,2),leg=false,ylim=(-0.02,0.02),title="Reconstruction of Y based \n on the first two modes",xlabel="x",ylabel="Yrecons")
    j+=1
end
gif(anim, "img/DMD_Yrecons_fps15.gif", fps = 15)

In [ ]:
plot(x,t,Yrecons',st=:surface,xaxis="x", yaxis="t",zaxis="Yrecons")
title!("Reconstruction first two modes")

## Same test function but with different parameters in order to mix alltogether the different scales

In [ ]:
A1 = 1.25
x1 = 0.25
σ1 = 0.4
f1 = 2.

A2 = 1.5
x2 = -0.25
σ2 = 0.2
f2 = 3.

#gaussian processes
yn1(x) = A1*exp(-((x-x1).^2)/(2*σ1^2))
yn2(x) = A2*exp(-((x-x2).^2)/(2*σ2^2));

In [ ]:
plot(x,yn1)
plot!(x,yn2,xlabel="x",ylabel="yn1,yn2")

In [ ]:
plot(t,sin.(2*π*f1*t))
plot!(t,sin.(2*π*f2*t),xlabel="t",ylabel="sin",xlim=(0,1),ylim=(-1,1))

In [ ]:
#  Building the two dimensional continous function f(x,t)
fn_1(x,t)=A1*exp(-((x-x1)^2)/(2*σ1^2))*sin(2*π*f1*t)
fn_2(x,t)=A2*exp(-((x-x2)^2)/(2*σ2^2))*sin(2*π*f2*t)
fn(x,t)=f_1(x,t)+f_2(x,t)

In [ ]:
anim=Animation()
anim = @animate for tt in range(0,1,length=100)
    plot( x, fn_1.(x,tt),xlim=(-2,2),title="fn_1 for one second",leg=false,ylim=(-2.,2.),xlabel="x",ylabel="f_1(x,t)")
end
gif(anim, "img/anim_fn_1_fps15.gif", fps = 15)

In [ ]:
anim=Animation()
anim = @animate for tt in range(0,1,length=100)
    plot( x, fn_2.(x,tt),xlim=(-2,2),title="fn_2 for one second",leg=false,ylim=(-2.,2.),xlabel="x",ylabel="f_2(x,t)")
end
gif(anim, "img/anim_fn_2_fps15.gif", fps = 15)

In [ ]:
anim=Animation()
anim = @animate for tt in range(0,1,length=100)
    plot( x, f.(x,tt),xlim=(-2,2),title="fn for one second",leg=false,ylim=(-2.,2.),xlabel="x",ylabel="f(x,t)")
end
gif(anim, "img/anim_fn_fps15.gif", fps = 15)

In [ ]:
YN = zeros(Nx,Nt)
j=1
for tt in t
    YN[:,j] = yn1.(x)*sin(2*π*f1*tt) + yn2.(x)*sin(2*π*f2*tt) # gaussian processes 
    j+=1                                           # with their temporal dynamics
end

In [ ]:
# Data decomposition with the singular value decomposition 
# U matrix (spatial Correlations); σ matrix (weighting of projections), Vt (time dynamics) 
Z=svd(YN)
#= plot of the singular values (see the  two most dominants features due to the two ranks dataset 
(the two bumps with their temporal dynamics). Again The very good decay shows the existence of a low rank subspace=#
plot(Z.S, yaxis=:log, xlim=(0,10),shape =:circle)

In [ ]:
#= Plot of the POD modes (Uj ; j={1,2})The POD modes are not so physical, 
a kind of blending of the bumps occurs due to the greedy nature of the SVD algorithm =#
plot(x,Z.U[:,1])
plot!(x,Z.U[:,2])
xlabel!("x")
ylabel!("u1(x), u2(x)")
title!("POD modes")

In [ ]:
#plot of the Mode coefficents (σjVtj ; j={1,2]})
plot(t,Z.S[1]*Z.Vt[1,:],xlim=(0, 1))
plot!(t,Z.S[2]*Z.Vt[2,:],xlim=(0, 1))
xlabel!("Time");ylabel!("a1(t), a2(t)");title!("Mode coefficients")

## Features Extraction (less good results no?)

In [ ]:
#Animation of the POD1 mode over the first second
YN_POD1=Z.U[:,1]*Z.S[1]*Z.Vt[1,:]'
anim=Animation()
j=1
anim = @animate for tt in range(0.01,1,length=100)
    plot( x, YN_POD1[:,j],leg=false,xlim=(-2,2),ylim=(-2.,2.),title="POD1 mode",xlabel="x",ylabel="Y_POD1")
    j+=1
end
gif(anim, "img/anim_YN_POD1_fps15.gif", fps = 15)

In [ ]:
#Animation of the POD2 mode over the first second
YN_POD2=Z.U[:,2]*Z.S[2]*Z.Vt[2,:]'
anim=Animation()
j=1
anim = @animate for tt in range(0.01,1,length=100)
    plot( x, YN_POD2[:,j],leg=false,xlim=(-2,2),ylim=(-2.,2.),title="POD2 mode",xlabel="x",ylabel="Y_POD2")
    j+=1
end
gif(anim, "img/anim_YN_POD2_fps15.gif", fps = 15)

In [ ]:
#Animation of Yrecons = Y_POD1+YPOD2 over the firt second
YNrecons=YN_POD1+YN_POD2
anim=Animation()
j=1
anim = @animate for tt in range(0.01,1,length=100)
    plot( x, YNrecons[:,j],xlim=(-2,2),leg=false,ylim=(-2.,2.),title="Reconstruction of Y based \n on the first two modes",xlabel="Space x",ylabel="Yrecons")
    j+=1
end
gif(anim, "img/anim_YNrecons_fps15.gif", fps = 15)

## What about  the dynamic mode decomposition (DMD) or robust DMD (rDMD) data-driven methods or even POD-Galerkin method coupled with the discrete empirical interpolation method (DEIM) for non linear reduction  